In [11]:
import numpy as np

In [12]:
#Simple 2D quadratic

def f1(x): 
    return ((x[0]-0.25)**2)+((x[1]-0.25)**2)

def gradf1(x):
    g = [2*(x[0]-0.25),2*(x[1]-0.25)]
    return np.asarray(g)

In [13]:
#General Initializations for simple Quadratic
w_init_q = np.array([0,1])
epsilon_q = 0.00001 #delta criteria
L1 = 2 #Lipschitz for quadratic
S1 = np.array([[-1,0],[1,0],[0,1]]) #Constrained Set Vertices

#Boosted FW initialization 

K = 10000 #theortically inf
T = 50 #max_iter

In [14]:
#Boosted FW for simple quadratic

def LPmin_b(x,S): #Check for vertex with min inner product with gradient of f(x)
    idx = np.argmin(np.matmul(S,gradf1(x)))
    return S[idx],idx

def max_IP(x,S): ##Frank Wolfe Step
    idx = np.argmax(np.matmul(S,x))
    return S[idx],idx

def proj_IP(r_k,v_k,x_k,d_k): #Ensure Convergence acc to the paper 
    if np.linalg.norm(d_k) != 0:
        if np.dot(r_k,v_k-x_k)>np.dot(r_k,-d_k/(np.linalg.norm(d_k))):
            return v_k-x_k
        else:
            return -d_k/(np.linalg.norm(d_k))
    else:
        return v_k-x_k

def align(d,d_hat): #Check for alignment
    if np.linalg.norm(d_hat) == 0:
        return -1
    else:
        return np.dot(d,d_hat)/(np.linalg.norm(d)*np.linalg.norm(d_hat))
    

def FW_boosted(y,K,delta,S,T,L):
    x_0,idx = LPmin_b(y,S)
    coordinates = []
    f_values = []
    itr = []
    Kt = K #Initialize with inf neighbors
    i = 0
    while i<=T: #Check for max iter
        d_0 = np.zeros(x_0.shape)
        lambda_t = 0
        for j in range(Kt):
            r_k = -gradf1(x_0)-d_0
            v_k,i1 = max_IP(r_k,S)
            u_k = proj_IP(r_k,v_k,x_0,d_0)
            lambda_k = np.dot(r_k,u_k)/(np.linalg.norm(u_k)**2)
            d_temp = d_0
            d_0 = d_0 + lambda_k*u_k
            if abs(align(-gradf1(x_0),d_temp)-align(-gradf1(x_0),d_0))>=delta:
                if np.array_equal(u_k,v_k-x_0):
                    lambda_t = lambda_t + lambda_k
                else:
                    lambda_t = lambda_t*(1-lambda_k/(np.linalg.norm(d_01)))
            else:
                Kt = j
                break
        g_t = d_0/lambda_t
        gamma_t = max(0,min(np.dot(-gradf1(x_0),g_t)/(L*(np.linalg.norm(g_t)**2)),1))
        x_0 = x_0 + gamma_t*g_t
        itr.append(i)
        coordinates.append(x_0)  
        f_values.append(f1(x_0))
        i = i+1
        if abs(np.dot(gradf1(x_0),(x_0-v_k)))<delta: #Convergence criteria
            endFlag = True
            break
    return coordinates,itr,f_values

In [15]:
coordinates3, itr3,f_values3 = FW_boosted(w_init_q,K,epsilon_q,S1,T,L1)

In [16]:
print("No of steps for convergence:",len(coordinates3))

No of steps for convergence: 2
